In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

white_path = "wine-white.csv"
red_path = "wine-red.csv"

red_whine = pd.read_csv(red_path)
white_whine = pd.read_csv(white_path)

white_whine['label'] = 'default'
red_whine['label'] = 'default'


In [3]:
def labelWhine(threshhold, whine_type, data): # function labels the white as being good or bad based on the threshhold value provided
    for index, row in data.iterrows():
        quality = row['quality']
        if quality<threshhold: 
            data.at[index,'label'] = "bad " + whine_type
        else:
            data.at[index,'label'] = "good " + whine_type
            
    return data


red_whine = labelWhine(6, "red", red_whine)  #labeling the whine as being good or bad for both white and red whites
white_whine = labelWhine(6, "white",white_whine)     

whine = pd.concat([red_whine,white_whine], ignore_index=True) # combining the red and white whine dataset after labeling 


            
whine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,label
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,bad red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,bad red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,bad red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,good red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,bad red
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,good white
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,bad white
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,good white
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,good white


In [9]:
dummies = pd.get_dummies(whine.label) # getting dummy values for one hot encoding
dummies

,bad red,bad white,good red,good white
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,0,0,1,0
4,1,0,0,0
...,...,...,...,...
6492,0,0,0,1
6493,0,1,0,0
6494,0,0,0,1
6495,0,0,0,1


In [12]:
merged = pd.concat([whine,dummies], axis = 'columns') # merging the "dummies" column with the whine dataset
merged

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,label,bad red,bad white,good red,good white
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,bad red,1,0,0,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,bad red,1,0,0,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,bad red,1,0,0,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,good red,0,0,1,0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,bad red,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,good white,0,0,0,1
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,bad white,0,1,0,0
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,good white,0,0,0,1
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,good white,0,0,0,1
